###SCD

In [0]:
from pyspark.sql.types import StructType, StringType, StructField
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import when



#### soruce dataframe

In [0]:

#  the schema for the DataFrame
schema = StructType([
    StructField("col1", StringType()),
    StructField("col2", StringType()),
    StructField("col3", StringType()),
    StructField("col4", StringType()),
    StructField("col5", StringType()),
     StructField("timestamp", StringType())
])

#test data generation
import datetime

data = [
    ('I', '1', '2', '3', '4'),
    ('D', '1', '2', '3', '4'),
    ('D', '4', '5', '6', '7'),
    ('u', '8', '9', '10', '12'),
    ('D', '8', '9', '10', '12'),
    ('I', '11', '12', '13', '14'),
    ('D', '', '', '', '')
]
import time 
def tiemfetch():
    timestamp = datetime.datetime.now()
    time.sleep(1)
    return timestamp



new_data = [(elem[0], elem[1], elem[2], elem[3], elem[4], str(tiemfetch()) )for elem in data]
print(new_data)

# incr  DataFrame
df_incr = spark.createDataFrame(new_data, schema)

df_incr.orderBy(df_incr.timestamp.asc()).display()

# Creating a window specification(only for demo purpose to server watermark/identification)
window_spec = Window.partitionBy().orderBy(df_incr["timestamp"].asc(),df_incr["col1"].asc())

df_incr = df_incr.withColumn("row_id", row_number().over(window_spec))
df_incr.display()

[('I', '1', '2', '3', '4', '2024-02-12 16:24:43.850171'), ('D', '1', '2', '3', '4', '2024-02-12 16:24:44.851150'), ('D', '4', '5', '6', '7', '2024-02-12 16:24:45.852206'), ('u', '8', '9', '10', '12', '2024-02-12 16:24:46.853262'), ('D', '8', '9', '10', '12', '2024-02-12 16:24:47.854316'), ('I', '11', '12', '13', '14', '2024-02-12 16:24:48.855368'), ('D', '', '', '', '', '2024-02-12 16:24:49.855798')]


col1,col2,col3,col4,col5,timestamp
I,1,2,3,4,2024-02-12 16:24:43.850171
D,1,2,3,4,2024-02-12 16:24:44.851150
D,4,5,6,7,2024-02-12 16:24:45.852206
u,8,9,10,12,2024-02-12 16:24:46.853262
D,8,9,10,12,2024-02-12 16:24:47.854316
I,11,12,13,14,2024-02-12 16:24:48.855368
D,,,,,2024-02-12 16:24:49.855798


col1,col2,col3,col4,col5,timestamp,row_id
I,1,2,3,4,2024-02-12 16:24:43.850171,1
D,1,2,3,4,2024-02-12 16:24:44.851150,2
D,4,5,6,7,2024-02-12 16:24:45.852206,3
u,8,9,10,12,2024-02-12 16:24:46.853262,4
D,8,9,10,12,2024-02-12 16:24:47.854316,5
I,11,12,13,14,2024-02-12 16:24:48.855368,6
D,,,,,2024-02-12 16:24:49.855798,7


In [0]:
from pyspark.sql.functions import lag, col,lead,when
from pyspark.sql.window import Window

# Create a new column 'prev_col1' with the value from the previous row in 'col1' column
windowSpec = Window.orderBy(col("timestamp"))
df_incr1 = df_incr.withColumn("prev_col1", lead("col1").over(windowSpec))



In [0]:
#null identification since it was said csv file
for col_name in df_incr1.dtypes:
    if (col_name[1] == 'string' or col_name[1]=='int' or col_name[1]=='date' or col_name[1]=='timestamp'):
        df_incr1=df_incr1.withColumn(col_name[0], when(
                (col(col_name[0]) == "") | (col(col_name[0]) == " ") | (col(col_name[0]) == '""')| (col(col_name[0]) == 'NULL'), None).otherwise(col(col_name[0])))
df_incr1.createOrReplaceTempView("inc_view")

#finding rows need to be update
row_id_deleted=spark.sql("""select  row_id
FROM inc_view
WHERE col2 IS NULL OR col3 IS NULL OR col4 IS NULL OR col5 IS NULL""").collect()
row_id_deleted=[i[0]-1 for i in row_id_deleted]


In [0]:

# Update col1 with prev_col1 for specific row_id
df_incr2 = df_incr1.withColumn("col1", when(col("row_id").isin(row_id_deleted), col("prev_col1")).otherwise(col("col1")))

# Display the updated DataFrame
df_incr2.display()
df_incr2.createOrReplaceTempView("incr_view")


col1,col2,col3,col4,col5,timestamp,row_id,prev_col1
I,1,2,3,4,2024-02-12 16:24:43.850171,1,D
D,1,2,3,4,2024-02-12 16:24:44.851150,2,D
D,4,5,6,7,2024-02-12 16:24:45.852206,3,u
u,8,9,10,12,2024-02-12 16:24:46.853262,4,D
D,8,9,10,12,2024-02-12 16:24:47.854316,5,I
D,11,12,13,14,2024-02-12 16:24:48.855368,6,D
D,null,null,null,null,2024-02-12 16:24:49.855798,7,null


In [0]:
source_table_name="incr_view"
spark.sql(f"drop table if exists table_tgt_temp")
spark.sql(
            f"""
            CREATE TABLE if not exists  table_tgt_temp (
                col1 CHAR(20),
                col2 CHAR(20),
                col3 CHAR(20),
                col4 CHAR(20),
                col5 CHAR(20)
            )
        """
        )
spark.sql(
    f"""
    MERGE INTO table_tgt_temp AS tgt
    USING (
        SELECT col1, col2, col3, col4, col5 from(

        select *,row_number() over (partition by col2,col3,col4,col5 order by timestamp desc) as final_rk from {source_table_name} where where col2 IS not NULL and col3 IS not NULL and col4 IS not NULL and col5 IS not NULL )final_view
        where final_view.final_rk=1
        
    ) AS src
    ON tgt.col2 = src.col2
    AND tgt.col3 = src.col3
    AND tgt.col4 = src.col4
    AND tgt.col5 = src.col5 
    WHEN MATCHED THEN
        UPDATE SET tgt.col1 = src.col1
    WHEN NOT MATCHED THEN
        INSERT (col1, col2, col3, col4, col5)
        VALUES (src.col1, src.col2, src.col3, src.col4, src.col5)
    """
)


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from table_tgt_temp

col1,col2,col3,col4,col5
D,1,2,3,4
D,11,12,13,14
D,4,5,6,7
D,8,9,10,12


In [0]:
%sql
select *,row_number() over (partition by col2,col3,col4,col5 order by timestamp desc) as final_rk from incr_view

col1,col2,col3,col4,col5,timestamp,row_id,prev_col1,final_rk
D,null,null,null,null,2024-02-12 16:24:49.855798,7,null,1
D,1,2,3,4,2024-02-12 16:24:44.851150,2,D,1
I,1,2,3,4,2024-02-12 16:24:43.850171,1,D,2
D,11,12,13,14,2024-02-12 16:24:48.855368,6,D,1
D,4,5,6,7,2024-02-12 16:24:45.852206,3,u,1
D,8,9,10,12,2024-02-12 16:24:47.854316,5,I,1
u,8,9,10,12,2024-02-12 16:24:46.853262,4,D,2
